In [ ]:
import os
import numpy as np
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, LeakyReLU, BatchNormalization

# Load MNIST data
(X_train, _), (_, _) = mnist.load_data()
X_train = (X_train.astype(np.float32) - 127.5) / 127.5
X_train = X_train.reshape(-1, 784)

# Generator
generator = Sequential([
    Dense(128, input_dim=100),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(256),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(512),
    LeakyReLU(0.2),
    BatchNormalization(),
    Dense(784, activation='tanh')
])

# Discriminator
discriminator = Sequential([
    Dense(512, input_dim=784),
    LeakyReLU(0.2),
    Dense(256),
    LeakyReLU(0.2),
    Dense(1, activation='sigmoid')
])

# Compile discriminator
discriminator.compile(loss='binary_crossentropy', optimizer='adam')

# Combined network
discriminator.trainable = False
gan_input = generator.input
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan.compile(loss='binary_crossentropy', optimizer='adam')

# Create directory if it doesn't exist
if not os.path.exists('generated_images'):
    os.makedirs('generated_images')

# Training
epochs, batch_size, half_batch = 10000, 64, 32

for epoch in range(epochs):
    idx = np.random.randint(0, X_train.shape[0], half_batch)
    real_images, noise = X_train[idx], np.random.normal(0, 1, (half_batch, 100))

    d_loss_real = discriminator.train_on_batch(real_images, np.ones((half_batch, 1)))
    d_loss_fake = discriminator.train_on_batch(generator.predict(noise), np.zeros((half_batch, 1)))
    d_loss = 0.5 * np.add(d_loss_real, d_loss_fake)

    gan_loss = gan.train_on_batch(noise, np.ones((half_batch, 1)))  # Adjusted labels to half_batch

    if epoch % 100 == 0:
        print(f"Epoch: {epoch}, D Loss: {d_loss}, G Loss: {gan_loss}")

    if epoch % 1000 == 0:
        img = generator.predict(np.random.normal(0, 1, (1, 100))).reshape(28, 28)
        plt.imshow(img, cmap='gray')
        plt.axis('off')
        plt.savefig(f"generated_images/gan_image_{epoch}.png")
        plt.close()


1/1 [==============================] - 0s 272ms/step
Epoch: 0, D Loss: 0.7675775289535522, G Loss: 0.5046956539154053
1/1 [==============================] - 0s 21ms/step
Epoch: 100, D Loss: 0.0020330134859989357, G Loss: 7.712738037109375
1/1 [==============================] - 0s 19ms/step
Epoch: 200, D Loss: 0.03771217726171017, G Loss: 16.908187866210938
1/1 [==============================] - 0s 20ms/step
Epoch: 300, D Loss: 0.05589892924763262, G Loss: 4.626543045043945
1/1 [==============================] - 0s 19ms/step
Epoch: 400, D Loss: 0.3180521259782836, G Loss: 9.355722427368164
1/1 [==============================] - 0s 22ms/step
Epoch: 500, D Loss: 0.22847946733236313, G Loss: 2.5004758834838867
1/1 [==============================] - 0s 22ms/step
Epoch: 600, D Loss: 0.10502013191580772, G Loss: 2.95906925201416
1/1 [==============================] - 0s 21ms/step
Epoch: 700, D Loss: 0.18787753582000732, G Loss: 3.153808116912842
1/1 [==============================] - 0s 21ms/